<a href="https://colab.research.google.com/github/akilaIduwara/SDGP_Project/blob/Akila_Induwara/SDGP_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Step 1: Import Required Libraries
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Step 2: Load and Preprocess Data
# Load data (replace with your actual file path)
try:
    df = pd.read_csv('/content/JWD_L8_DD_ERTmodel.csv', skiprows=1, header=0)

    # Strip spaces from column names
    df.columns = df.columns.str.strip()

    # Print column names to verify
    print("Columns in the CSV file:", df.columns.tolist())

    # Check if 'Log10Res' column exists
    if 'Log10Res' not in df.columns:
        raise KeyError("Column 'Log10Res' not found in the CSV file. Please check the column names.")

    # Clean invalid resistivity values (remove rows where Log10Res is -9999)
    df = df[df['Log10Res'] != -9999]

    # Convert Log10Res to resistivity (ohm-m)
    df['Resistivity'] = 10 ** df['Log10Res']

    # Create target variable: 1 if resistivity is between 20 and 100 ohm-m, else 0
    df['is_water'] = np.where((df['Resistivity'] >= 20) & (df['Resistivity'] <= 100), 1, 0)

    # Extract features (X, Y, Elev, Dist) and target (is_water)
    features = df[['Dist', 'X_NAD83UTMz16N', 'Y_NAD83UTMz16N', 'Elev']]
    target = df['is_water']

except Exception as e:
    print("Error during data loading or preprocessing:", e)
    exit()

# Step 3: Train XGBoost Model
try:
    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(
        features, target, test_size=0.2, random_state=42
    )

    # Initialize and train XGBoost classifier
    model = xgb.XGBClassifier(
        objective='binary:logistic',
        eval_metric='logloss'
    )
    model.fit(X_train, y_train)

    # Evaluate model
    y_pred = model.predict(X_test)
    print("Classification Report:")
    print(classification_report(y_test, y_pred))

except Exception as e:
    print("Error during model training or evaluation:", e)
    exit()

# Step 4: Identify Water Resources and Display Results
try:
    # Predict water locations in the entire dataset
    df['predicted_water'] = model.predict(features)

    # Filter rows where water is predicted
    water_locations = df[df['predicted_water'] == 1]

    # Display results (Depth = absolute value of Elev)
    results = water_locations[[
        'X_NAD83UTMz16N', 'Y_NAD83UTMz16N', 'Elev'
    ]].rename(columns={'Elev': 'Depth'})

    results['Depth'] = np.abs(results['Depth'])  # Convert elevation to depth

    print("\nIdentified Water Resources:")
    print(results.head(10).to_string(index=False, justify='left'))  # Display formatted output

    # Save results to CSV
    results.to_csv("identified_water_resources.csv", index=False)
    print("Results saved to identified_water_resources.csv")

except Exception as e:
    print("Error during prediction or result display:", e)
    exit()


Columns in the CSV file: ['Dist', 'X_NAD83UTMz16N', 'Y_NAD83UTMz16N', 'Elev', 'Log10Res', 'Shading']
Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.98      0.98     65625
           1       0.99      0.99      0.99    196173

    accuracy                           0.99    261798
   macro avg       0.98      0.98      0.98    261798
weighted avg       0.99      0.99      0.99    261798


Identified Water Resources:
 X_NAD83UTMz16N  Y_NAD83UTMz16N  Depth
703846.44       3398898.58      35.53 
703846.66       3398898.68      35.33 
703846.66       3398898.68      35.43 
703846.66       3398898.68      35.53 
703846.66       3398898.68      35.63 
703846.89       3398898.78      35.13 
703846.89       3398898.78      35.23 
703846.89       3398898.78      35.33 
703846.89       3398898.78      35.43 
703846.89       3398898.78      35.53 
Results saved to identified_water_resources.csv
